In [1]:
import pandas as pd
import glob, os

file = "/Users/sagardhal/Desktop/Practice/personal-stock/data/stock_data_combined_20250904_071145.parquet"
# load the dataframe
df = pd.read_parquet(file)

print("Shape:", df.shape)
print("First 5 columns:", df.columns[:5].tolist())

# look for macro-ish names
macro_cols = [c for c in df.columns if any(k in c.lower() for k in ["macro", "dgs", "fedfunds", "fed"])]
print("\nMacro-like columns found:", len(macro_cols))
print(macro_cols[:50])  # show first 50 if there are many


Shape: (2094565, 222)
First 5 columns: ['Date', 'Open', 'High', 'Low', 'Close']

Macro-like columns found: 8
['fedfunds_yoy', 'fedfunds_qoq', 'dgs1_yoy', 'dgs1_qoq', 'dgs5_yoy', 'dgs5_qoq', 'dgs10_yoy', 'dgs10_qoq']


In [ ]:
df.head()

,Date,Open,High,Low,Close,adj_close,Volume,Ticker,year,month,...,cpilfesl_yoy,cpilfesl_qoq,fedfunds_yoy,fedfunds_qoq,dgs1_yoy,dgs1_qoq,dgs5_yoy,dgs5_qoq,dgs10_yoy,dgs10_qoq
0,1980-12-12,0.128348,0.128906,0.128348,0.128348,0.098485,469033600.0,AAPL,1980,12,...,0.121532,0.034105,0.371553,0.73873,0.295359,0.315338,0.295627,0.171969,0.260496,0.146701
1,1980-12-15,0.122210,0.122210,0.121652,0.121652,0.093347,175884800.0,AAPL,1980,12,...,0.121532,0.034105,0.371553,0.73873,0.320643,0.326253,0.304554,0.172208,0.266730,0.145203
2,1980-12-16,0.113281,0.113281,0.112723,0.112723,0.086495,105728000.0,AAPL,1980,12,...,0.121532,0.034105,0.371553,0.73873,0.343486,0.360754,0.337143,0.194894,0.297791,0.171726
3,1980-12-17,0.115513,0.116071,0.115513,0.115513,0.088636,86441600.0,AAPL,1980,12,...,0.121532,0.034105,0.371553,0.73873,0.341880,0.382042,0.336224,0.202949,0.285576,0.172109
4,1980-12-18,0.118862,0.119420,0.118862,0.118862,0.091206,73449600.0,AAPL,1980,12,...,0.121532,0.034105,0.371553,0.73873,0.341880,0.295833,0.336224,0.156879,0.285576,0.130881


In [4]:
df.describe().T

/Users/sagardhal/Desktop/Practice/personal-stock/.venv/lib/python3.13/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/Users/sagardhal/Desktop/Practice/personal-stock/.venv/lib/python3.13/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,count,mean,min,25%,50%,75%,max,std
Date,2094565,2004-10-30 11:53:54.755283456,1962-01-02 00:00:00,1994-10-06 00:00:00,2007-04-20 00:00:00,2017-01-10 00:00:00,2025-09-03 00:00:00,NaN
Open,2094541.0,64.077145,0.005208,7.76,25.75,62.810001,5815.0,153.689353
High,2094541.0,64.837645,0.005208,7.88,26.1,63.540001,5839.410156,155.444593
Low,2094541.0,63.299287,0.004801,7.64855,25.389999,62.088051,5707.930176,151.902561
Close,2094541.0,64.086593,0.005208,7.765625,25.75,62.84,5815.919922,153.730653
...,...,...,...,...,...,...,...,...
dgs1_qoq,2094503.0,0.057383,-0.930818,-0.098696,0.0,0.109974,5.357143,0.450129
dgs5_yoy,2094320.0,0.11295,-0.869955,-0.196078,-0.010043,0.202216,3.818182,0.602092
dgs5_qoq,2094503.0,0.020363,-0.791411,-0.090379,0.003012,0.089744,1.5,0.218543
dgs10_yoy,2094320.0,0.043137,-0.787402,-0.154206,-0.015267,0.156895,1.836066,0.347941


In [6]:
# ---- Group by ticker and get min/max ----
date_ranges = (
    df.groupby("Ticker")["Date"]
      .agg(Start="min", End="max")
      .reset_index()
)

# ---- Add number of days covered ----
date_ranges["Days"] = (date_ranges["End"] - date_ranges["Start"]).dt.days + 1

date_ranges = date_ranges.sort_values("Start", ascending=False)


display(date_ranges.head(20))  # show first 20
print(f"\nTotal tickers: {len(date_ranges)}")

,Ticker,Start,End,Days
64,CVS,2025-07-31,2025-09-03,35
96,GEV,2024-03-27,2025-09-03,526
22,ARM,2023-09-14,2025-09-03,721
130,KVUE,2023-05-04,2025-09-03,854
46,CEG,2022-01-19,2025-09-03,1324
21,APP,2021-04-15,2025-09-03,1603
55,COIN,2021-04-14,2025-09-03,1604
2,ABNB,2020-12-10,2025-09-03,1729
67,DASH,2020-12-09,2025-09-03,1730
187,PLTR,2020-09-30,2025-09-03,1800



Total tickers: 250


In [8]:
coverage = df.groupby("Ticker")["Date"].agg(Start="min", End="max", Days="count")

coverage


,Start,End,Days
Ticker,,,
AAPL,1980-12-12,2025-09-03,11266
ABBV,2013-01-02,2025-09-03,3181
ABNB,2020-12-10,2025-09-03,1181
ABT,1980-03-17,2025-09-03,11454
ACN,2001-07-19,2025-09-03,6061
...,...,...,...
WMT,1972-08-25,2025-09-03,13360
XOM,1962-01-02,2025-09-03,16019
XYZ,2015-11-19,2025-09-03,2454


In [10]:
nulls = df.isnull().sum().sort_values(ascending=False)
nulls_by_ticker = df.groupby("Ticker").apply(lambda x: x.isnull().sum())
nulls_by_ticker


/var/folders/z9/l3h5t0t97_9cn2kzxx156g2m0000gn/T/ipykernel_95755/2688343947.py:2: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  nulls_by_ticker = df.groupby("Ticker").apply(lambda x: x.isnull().sum())


,Date,Open,High,Low,Close,adj_close,Volume,Ticker,year,month,...,cpilfesl_yoy,cpilfesl_qoq,fedfunds_yoy,fedfunds_qoq,dgs1_yoy,dgs1_qoq,dgs5_yoy,dgs5_qoq,dgs10_yoy,dgs10_qoq
Ticker,,,,,,,,,,,,,,,,,,,,,
AAPL,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ABBV,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ABNB,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ABT,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ACN,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WMT,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
XOM,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
XYZ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
bad_prices = df[(df["Close"] <= 0) | (df["Open"] <= 0)]
low_volume = df.groupby("Ticker")["Volume"].apply(lambda v: (v == 0).mean())

low_volume


Ticker
AAPL    0.000089
ABBV    0.000000
ABNB    0.000000
ABT     0.000000
ACN     0.000000
          ...   
WMT     0.003967
XOM     0.000000
XYZ     0.000000
ZS      0.000000
ZTS     0.000000
Name: Volume, Length: 250, dtype: float64

In [13]:
df["Return"] = df.groupby("Ticker")["Close"].pct_change()
outliers = df[df["Return"].abs() > 0.5]
outliers

/var/folders/z9/l3h5t0t97_9cn2kzxx156g2m0000gn/T/ipykernel_95755/3013944322.py:1: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df["Return"] = df.groupby("Ticker")["Close"].pct_change()


,Date,Open,High,Low,Close,adj_close,Volume,Ticker,year,month,...,cpilfesl_qoq,fedfunds_yoy,fedfunds_qoq,dgs1_yoy,dgs1_qoq,dgs5_yoy,dgs5_qoq,dgs10_yoy,dgs10_qoq,Return
5004,2000-09-29,0.503348,0.517857,0.453125,0.459821,0.386509,7.421641e+09,AAPL,2000,9,...,0.006626,0.249042,-0.001531,0.117864,0.003295,-0.031457,-0.035948,-0.052288,-0.030000,-0.518692
41665,2016-04-22,3.190000,3.990000,3.180000,3.990000,3.990000,1.432653e+08,AMD,2016,4,...,0.006515,2.083333,0.088235,1.240000,0.191489,-0.132911,-0.055172,-0.160000,-0.059701,0.522901
100948,2008-11-24,61.200001,65.000000,52.500000,59.500000,44.306896,8.037831e+07,C,2008,11,...,0.002560,-0.913140,-0.805000,-0.703125,-0.560185,-0.339233,-0.258278,-0.166667,-0.111406,0.578249
294053,2000-03-20,109.250000,119.000000,85.750000,86.750000,86.750000,1.732560e+07,MSTR,2000,3,...,0.007275,0.216216,0.103774,0.317125,0.043551,0.252427,0.022187,0.172676,-0.032864,-0.617420
294326,2001-04-19,3.460000,7.170000,3.390000,5.240000,5.240000,1.004660e+07,MSTR,2001,4,...,0.006525,-0.202658,-0.197324,-0.367576,-0.177453,-0.263636,-0.002053,-0.166139,0.003810,0.764310
646093,1968-05-21,0.427984,0.427984,0.413580,0.422840,0.177806,2.515050e+06,MCD,1968,5,...,0.008403,0.553299,0.299363,0.525060,0.168207,0.312896,0.108108,0.248963,0.075812,-0.507194
646334,1969-06-13,0.275720,0.275720,0.265432,0.267490,0.112789,1.810350e+06,MCD,1969,6,...,0.013228,0.466227,0.310751,0.169748,0.099526,0.146299,0.034161,0.154255,0.025197,-0.500959
704155,2022-03-16,37.119999,42.970001,35.639999,42.619999,42.619999,7.970770e+07,PDD,2022,3,...,0.013404,1.857143,1.500000,21.500000,4.000000,1.449438,0.847458,0.265896,0.553191,0.560601
772728,2015-05-21,2.800000,2.874000,2.411000,2.568000,2.568000,1.230390e+08,SHOP,2015,5,...,0.006312,0.333333,0.090909,1.200000,0.000000,-0.072727,-0.019231,-0.157692,0.063107,0.510588
835824,2012-05-07,52.119999,58.750000,51.849998,58.119999,58.119999,4.021630e+07,VRTX,2012,5,...,0.005219,0.777778,0.600000,-0.052632,0.200000,-0.572973,-0.036585,-0.394322,-0.044776,0.553595
